### This notebook launches the LatissAcquireAndTakeSequence SAL script, meant for the scriptQueue, but via a notebook.

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-26495**

In [ ]:
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import astropy

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

In [ ]:
# for autocomplete to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

## Emulate how the scriptQueue launches scripts
### Restart from here if re-launching a script

In [ ]:
script = LatissAcquireAndTakeSequence(index=1)  # this essentially calls the init method of the script
# make sure all remotes etc are running
await script.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

In [ ]:
configuration = yaml.safe_dump({"object_name": 'HIP 31636',
                                "do_acquire": False,
                                "acq_filter" : 'BG40',
                                "acq_grating" : 'empty_1',
                                "exposure_time_sequence" : [4, 4, 1, 2, 1, 2], 
                                "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
                                "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
                                "doPointingModel": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "max_acq_iter": 3,
                                "target_pointing_verification": True})
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

run_data = script.cmd_run.DataType()
await script.do_run(run_data)
await script.done_task